In [6]:
import os

os.getcwd()

'd:\\meli_case_solution\\notebooks'

In [ ]:
import os



In [1]:
import requests
cat_id = 'MLA1000'
offset = 50
url = f'https://api.mercadolibre.com/sites/MLA/search?category={cat_id}&offset={offset}'
request = requests.get(url)
items = request.json()


In [4]:
import pandas as pd

pd.DataFrame(items['results']).head()

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,sanitized_title,permalink,buying_mode,site_id,...,winner_item_id,catalog_listing,discounts,promotion_decorations,promotions,inventory_id,official_store_name,variation_id,variation_filters,variations_data
0,MLA1990948038,Auriculares Inalámbricos Xiaomi Redmi Buds 6 P...,new,833941-MLU78764933573_082024,MLA39962085,gold_special,,https://www.mercadolibre.com.ar/auriculares-in...,buy_it_now,MLA,...,None,True,None,None,None,None,NaN,NaN,NaN,NaN
1,MLA1901528344,Smart Tv Hyundai Hyled-32fhd7a 32'' Hd Led,new,626511-MLA80038416778_102024,MLA19800475,gold_pro,,https://www.mercadolibre.com.ar/smart-tv-hyund...,buy_it_now,MLA,...,None,True,None,None,None,FBOU05094,Mercado Libre,NaN,NaN,NaN
2,MLA1693932190,Auriculares Inalámbricos Jd Air Free In Ear Bl...,new,798175-MLU74507462869_022024,MLA32823373,gold_special,,https://www.mercadolibre.com.ar/auriculares-in...,buy_it_now,MLA,...,None,True,None,None,None,QHYZ69235,NaN,NaN,NaN,NaN
3,MLA1610631786,Pila Aa Scp Recargables Ni-mh-2700 Cilíndrica ...,new,659587-MLA79342019892_092024,MLA16998583,gold_special,,https://www.mercadolibre.com.ar/pila-aa-scp-re...,buy_it_now,MLA,...,None,True,None,None,None,EQYA89289,NaN,NaN,NaN,NaN
4,MLA1983261646,Amazon Alexa Echo Dot 5ta Generación Parlante ...,new,858077-MLU73903942149_012024,MLA29588132,gold_special,,https://www.mercadolibre.com.ar/amazon-alexa-e...,buy_it_now,MLA,...,None,True,None,None,None,IRRL64125,NaN,NaN,NaN,NaN


In [19]:
from pathlib import Path

root_path = Path('d:/meli_case_solution')

In [20]:
import pandas as pd
import yaml
import requests
from concurrent.futures import ThreadPoolExecutor

with open(os.path.join(root_path, 'src', 'config', 'country_infos.yaml'), "r") as file:
        country_info = yaml.safe_load(file)

In [58]:
offsets = [i for i in range(0, 1000, 50)]

def fetch_category_data(country_id, category_id):
    serial_lst = []
    with requests.Session() as session:
        for offset in offsets:
            url = f"https://api.mercadolibre.com/sites/{country_id}/search?category={category_id}&offset={offset}"
            response = session.get(url)
            items = response.json()
            serial_lst.append(pd.DataFrame(items['results']))
    return pd.concat(serial_lst, ignore_index=True)


def process_country(country):
    print(f"{country['id']}:")
    cats = requests.get(f"https://api.mercadolibre.com/sites/{country['id']}/categories")
    category_ids_country = {item['id'] for item in cats.json()}

    with ThreadPoolExecutor() as executor:
        dataframes = list(executor.map(lambda category_id: fetch_category_data(country['id'], category_id), category_ids_country))

    df_country_id = pd.concat(dataframes, ignore_index=True)
    df_country_id.drop(columns=['variations_data'], axis=1, inplace=True)

    return df_country_id



In [64]:
# df = process_country(country_info['country'][0])
df1 = process_country(country_info['country'][1])
#df_ar = process_country(country_info['country'][2])

MRD:


In [65]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10757 entries, 0 to 10756
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     10757 non-null  object
 1   title                  10757 non-null  object
 2   condition              9773 non-null   object
 3   thumbnail_id           10757 non-null  object
 4   catalog_product_id     382 non-null    object
 5   listing_type_id        10757 non-null  object
 6   sanitized_title        10757 non-null  object
 7   permalink              10757 non-null  object
 8   buying_mode            10757 non-null  object
 9   site_id                10757 non-null  object
 10  category_id            10757 non-null  object
 11  domain_id              10757 non-null  object
 12  thumbnail              10757 non-null  object
 13  currency_id            10174 non-null  object
 14  order_backend          10757 non-null  int64 
 15  price              

In [82]:
df1.attributes.iloc[10755]

[{'id': 'BRAND',
  'name': 'Marca',
  'value_id': '1089',
  'value_name': 'Hyundai',
  'attribute_group_id': 'FIND',
  'attribute_group_name': 'Ficha técnica',
  'value_struct': None,
  'values': [{'id': '1089',
    'name': 'Hyundai',
    'struct': None,
    'source': 7044766375017495}],
  'source': 7044766375017495,
  'value_type': 'string'},
 {'id': 'COLOR',
  'name': 'Color',
  'value_id': '52055',
  'value_name': 'Blanco',
  'attribute_group_id': 'FIND',
  'attribute_group_name': 'Ficha técnica',
  'value_struct': None,
  'values': [{'id': '52055',
    'name': 'Blanco',
    'struct': None,
    'source': 7044766375017495}],
  'source': 7044766375017495,
  'value_type': 'string'},
 {'id': 'KILOMETERS',
  'name': 'Kilómetros',
  'value_id': None,
  'value_name': '97050 km',
  'attribute_group_id': 'FIND',
  'attribute_group_name': 'Ficha técnica',
  'value_struct': {'number': 97050, 'unit': 'km'},
  'values': [{'id': None,
    'name': '97050 km',
    'struct': {'number': 97050, 'unit'

In [26]:
for col in df_ar.columns:
    df_ar[col] = df_ar[col].apply(lambda x: str(x) if isinstance(x, dict) else x)

In [ ]:
# path_output = os.path.join(root_path, 'data', 'raw', 'df_ar_raw.parquet')

# df_ar.to_parquet(path_output, index=False, engine='pyarrow')

In [37]:
def check_struct_in_columns(df):
    struct_columns = []
    
    for col in df.columns:
        # Verifica o tipo da coluna
        col_type = df[col].apply(type).iloc[2]  # Verifica o tipo do primeiro item na coluna
        if isinstance(col_type, dict):
            struct_columns.append(col)
    
    return struct_columns

# Verificar as colunas com tipo struct (ou dict)
struct_columns = check_struct_in_columns(df_ar)
print(f"Colunas com tipo struct/dict: {struct_columns}")

Colunas com tipo struct/dict: []


In [62]:
df = pd.read_parquet(path_output)

In [ ]:
df1.seller_contact.iloc[3].get('contact')

str

In [53]:
df_ar.iloc[1]

id                                                           MLA1783673866
title                             Antihumedad Aparato+repuesto Mini X3 Uni
condition                                                              new
thumbnail_id                                  924152-MLU74219028695_012024
catalog_product_id                                             MLA19688973
listing_type_id                                               gold_special
sanitized_title                                                           
permalink                https://www.mercadolibre.com.ar/antihumedad-ap...
buying_mode                                                     buy_it_now
site_id                                                                MLA
category_id                                                       MLA30791
domain_id                                               MLA-AIR_FRESHENERS
variation_id                                                           NaN
thumbnail                

In [57]:
df_ar.variations_data.iloc[0]

{'180670362339': {'thumbnail': 'http://http2.mlstatic.com/D_721198-MLA72827750058_112023-O.jpg',
  'ratio': '0.64',
  'name': '',
  'pictures_qty': 5,
  'price': 4768.94,
  'inventory_id': 'JFIV14360',
  'user_product_id': 'MLAU140002964',
  'attributes': [{'id': 'GTIN',
    'name': 'Código universal de producto',
    'value_name': 'string',
    'value_type': '7798184686822'}],
  'attribute_combinations': []},
 '180670362381': {'thumbnail': 'http://http2.mlstatic.com/D_963781-MLA72904344905_112023-O.jpg',
  'ratio': '0.63',
  'name': '',
  'pictures_qty': 5,
  'price': 4768.94,
  'inventory_id': 'GBGD92940',
  'user_product_id': 'MLAU140012340',
  'attributes': [{'id': 'GTIN',
    'name': 'Código universal de producto',
    'value_name': 'string',
    'value_type': '7798184683869'}],
  'attribute_combinations': []},
 '180670362365': {'thumbnail': 'http://http2.mlstatic.com/D_619094-MLA73445399181_122023-O.jpg',
  'ratio': '0.63',
  'name': '',
  'pictures_qty': 5,
  'price': 4768.94,
 

In [63]:
df.sale_price.apply(lambda x: x.get('type', None)).unique()

array(['promotion', 'standard', 'BNSP', 'SPONTH', 'BNGM'], dtype=object)

In [11]:
df1.shape

(10606, 39)

In [13]:
cols_in_df_not_in_df1 = set(df.columns) - set(df1.columns)
cols_in_df1_not_in_df = set(df1.columns) - set(df.columns)

print(list(cols_in_df1_not_in_df | cols_in_df_not_in_df1))

['official_store_name']


In [70]:
len([items['results'][i] for i in range(len(items['results']))])

43

In [ ]:
df = pd.json_normalize(items['results'], max_level=1, sep='__')
df.head()

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,sanitized_title,permalink,buying_mode,site_id,...,installments__rate,installments__currency_id,installments__metadata,official_store_name,installments,variation_id,variation_filters,variations_data__181097386212,variations_data__181097386214,variations_data__181097386216
0,MLA1792714848,Smart Tv Philips 50pud7408/77 50 4k Led,new,989321-MLU74192118913_012024,MLA28130191,gold_special,,https://www.mercadolibre.com.ar/smart-tv-phili...,buy_it_now,MLA,...,15.61,ARS,"{'meliplus_installments': False, 'additional_b...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MLA1720389490,Kit 2 Microfonos Inalambricos Boya By-v20 2.4g...,new,600464-MLB53497379572_012023,MLA22622348,gold_pro,,https://www.mercadolibre.com.ar/kit-2-microfon...,buy_it_now,MLA,...,0.00,ARS,"{'meliplus_installments': False, 'additional_b...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MLA1772790178,Tocadiscos Winco Wincofon W408 Vinilo Vintage ...,new,702402-MLA41241337976_032020,MLA15544119,gold_special,,https://www.mercadolibre.com.ar/tocadiscos-win...,buy_it_now,MLA,...,34.86,ARS,"{'meliplus_installments': False, 'additional_b...",Electro Point,NaN,NaN,NaN,NaN,NaN,NaN
3,MLA1928589912,4 Pilas Aaa Energizer Max E92 Alcalina,new,847664-MLU78281559235_082024,MLA16122092,gold_special,,https://www.mercadolibre.com.ar/4-pilas-aaa-en...,buy_it_now,MLA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MLA1679550160,Radio Portatil Retro Vintage Spica Sp-590 Am/f...,new,708707-MLU74678301787_022024,MLA34018049,gold_special,,https://www.mercadolibre.com.ar/radio-portatil...,buy_it_now,MLA,...,34.86,ARS,"{'meliplus_installments': False, 'additional_b...",LIBERCAM,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
print(len(df.columns))
df.columns

65


Index(['id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id',
       'listing_type_id', 'sanitized_title', 'permalink', 'buying_mode',
       'site_id', 'category_id', 'domain_id', 'thumbnail', 'currency_id',
       'order_backend', 'price', 'original_price', 'available_quantity',
       'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago',
       'stop_time', 'attributes', 'winner_item_id', 'catalog_listing',
       'discounts', 'promotion_decorations', 'promotions', 'inventory_id',
       'sale_price__price_id', 'sale_price__amount', 'sale_price__conditions',
       'sale_price__currency_id', 'sale_price__exchange_rate',
       'sale_price__payment_method_prices', 'sale_price__payment_method_type',
       'sale_price__regular_amount', 'sale_price__type',
       'sale_price__metadata', 'shipping__store_pick_up',
       'shipping__free_shipping', 'shipping__logistic_type', 'shipping__mode',
       'shipping__tags', 'shipping__benefits', 'shipping__promise',
       

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 99 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   id                                                     50 non-null     object 
 1   title                                                  50 non-null     object 
 2   condition                                              50 non-null     object 
 3   thumbnail_id                                           50 non-null     object 
 4   catalog_product_id                                     49 non-null     object 
 5   listing_type_id                                        50 non-null     object 
 6   sanitized_title                                        50 non-null     object 
 7   permalink                                              50 non-null     object 
 8   buying_mode                                         

In [40]:
df[df.filter(like='variations_data').columns]

,variations_data__181097386212__thumbnail,variations_data__181097386212__ratio,variations_data__181097386212__name,variations_data__181097386212__pictures_qty,variations_data__181097386212__price,variations_data__181097386212__user_product_id,variations_data__181097386212__attributes,variations_data__181097386212__attribute_combinations,variations_data__181097386214__thumbnail,variations_data__181097386214__ratio,...,variations_data__181097386214__attributes,variations_data__181097386214__attribute_combinations,variations_data__181097386216__thumbnail,variations_data__181097386216__ratio,variations_data__181097386216__name,variations_data__181097386216__pictures_qty,variations_data__181097386216__price,variations_data__181097386216__user_product_id,variations_data__181097386216__attributes,variations_data__181097386216__attribute_combinations
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
